In [1]:
from joblib import load
import PersonCreator

In [2]:
clf = load('random_forest.joblib')

PersonCreator.display_creator()

Output()

Output()

Output()

Output()

Output()

In [6]:
person = PersonCreator.create().reshape(1, -1)
print(person)
print(clf.predict(person))

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])
[0.]
